# Exploratory Data Analysis, Hop Teaming Dataset

### Import libraries, connect to the database, and take a look around

In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [50]:
# Set the display to show more rows and columns
pd.options.display.max_rows = 500
pd.options.display.max_columns = 200

In [2]:
# Assign sqlite database to db variable
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [3]:
# Check the tables in the database
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in tables:
    print(table[0])

cbsa
specialty
referrals
nashville_referrals
nppes


In [39]:
# For reference: pull out the column names for a given table
col = db.execute("PRAGMA table_info(specialty)").fetchall()
print([e[1] for e in col])

['npi', 'primary_taxonomy', 'Classification']


### How many providers are in the nppes dataset?
*Note that `entity_type_code` 1 = Providers and `entity_type_code` 2 = Facilities.*

In [18]:
# Create a query to get all providers with a practice location within the Nashville CBSA
nash_providers = """
    SELECT n.*

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
"""

# Read the query results into a dataframe
nash_nppes = pd.read_sql(nash_providers, db)

In [21]:
# Take a look at the number of datapoints available
nash_nppes.info()
# There are 38,414 rows in the table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38414 entries, 0 to 38413
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   npi                         38414 non-null  int64  
 1   entity_type_code            38414 non-null  float64
 2   provider_organization_name  7574 non-null   object 
 3   provider_last_name          30839 non-null  object 
 4   provider_first_name         30840 non-null  object 
 5   provider_middle_name        19914 non-null  object 
 6   provider_name_prefix        11560 non-null  object 
 7   provider_name_suffix        907 non-null    object 
 8   provider_credential         25323 non-null  object 
 9   address_01                  38414 non-null  object 
 10  address_02                  9957 non-null   object 
 11  city                        38414 non-null  object 
 12  state                       38414 non-null  object 
 13  zip_9                       384

In [34]:
# Take a look at the unique provider credential types
nash_nppes.provider_credential.unique().tolist()

['CRNP',
 'M.D.',
 'MD',
 'FNP',
 None,
 'M. D.',
 'DO',
 'PHARMD',
 'PA',
 'D.M.D.',
 'RN, APN',
 'DDS',
 'CRNA, MS',
 'NP',
 'APN',
 'APRN',
 'C.R.N.A., A.P.N.',
 'FNP-C',
 'D.C.',
 'RPH, PHARMD,MS',
 'O.D.',
 'P.T., D.P.T',
 'D.D.S.',
 'N.P.',
 'APRN, BC, FNP',
 'PAC',
 'ACNP',
 'M.S.S.W.',
 'GNP',
 'MD, PHD',
 'MSN APRN-BC',
 'DPM',
 'PHARM. D.',
 'PH.D.',
 'CRNA',
 'NP-C',
 'PA-C',
 'PHARM.D.',
 'APRN, BC',
 'O.D',
 'M.D., PH.D.',
 'D.D.S',
 'LCSW',
 'LCSW, ACSW',
 'L.C.S.W.',
 'DDS, MD',
 'DO, MBA',
 'DC',
 'M.D., R.V.T',
 'ARNP',
 'PT',
 'D.O. F.A.C.O.G.',
 'DMD',
 'M.S., P.A.-C.',
 'D.O.',
 'MS, PT',
 'P.A-C..',
 'CNM',
 'MSPT',
 'DO, RDMS',
 'P.T.',
 'PHD CCC-SLP',
 'DPT',
 'DP',
 'D.P.T.',
 'PTA',
 'COTA',
 'O.T.',
 'D.PH.',
 'PHD, FNP- BC',
 'DNP,FNP- BC',
 'MD MBA',
 'D.P.M.',
 'ACNP-BC',
 'LDN',
 'MSN, APRN-BC, CNN',
 'DNP',
 'OD',
 'FNP C',
 'ED.D.',
 'ANPC',
 'MSSW, LCSW',
 'MD, MS, FIPP',
 'M.D., MPH',
 'DDS,PC',
 'WHNP CNM',
 'MSN',
 'CFNP',
 'PSY.D.',
 'ANP-C',
 'P.A.

*The credentials are VERY not-standardized and include multiple variants of the same thing with different spellings (e.g. `PharmD` vs. `Pharm.D.` vs. `PHARMD`) in addition to commas within the field itself (e.g. `APN-BC, NP-C`). If we need to use this field, we will need to do some data cleaning.*

In [35]:
# Subset to only individual providers
ind_nash_nppes = nash_nppes[nash_nppes['entity_type_code'] == 1]

# Take a look at how many individual providers there are in the Nashville area
ind_nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30840 entries, 0 to 38413
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   npi                         30840 non-null  int64  
 1   entity_type_code            30840 non-null  float64
 2   provider_organization_name  0 non-null      object 
 3   provider_last_name          30839 non-null  object 
 4   provider_first_name         30840 non-null  object 
 5   provider_middle_name        19914 non-null  object 
 6   provider_name_prefix        11560 non-null  object 
 7   provider_name_suffix        907 non-null    object 
 8   provider_credential         25323 non-null  object 
 9   address_01                  30840 non-null  object 
 10  address_02                  7556 non-null   object 
 11  city                        30840 non-null  object 
 12  state                       30840 non-null  object 
 13  zip_9                       308

In [36]:
# Who is the provider without a last name?
ind_nash_nppes[ind_nash_nppes['provider_last_name'].isna() == True]

npi  entity_type_code provider_organization_name  \
10346  1215064852               1.0                       None   

      provider_last_name provider_first_name provider_middle_name  \
10346               None           ELIZABETH                  ANN   

      provider_name_prefix provider_name_suffix provider_credential  \
10346                  DR.                 None                M.D.   

              address_01 address_02      city state        zip_9  zip_5  
10346  1222 TROTWOOD AVE  SUITE 101  COLUMBIA    TN  384016436.0  38401

*Hilariously, [this is a real provider](https://npiregistry.cms.hhs.gov/registry/provider-view/1215064852) whose last name is "Null".*

### Explore the specialties data

In [41]:
specialties = """
    SELECT n.*
        , s.primary_taxonomy
        , s.Classification

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
    
    --WHERE entity_type_code = 1 -- only get individuals
"""
# Read the query results into a dataframe
nash_nppes = pd.read_sql(specialties, db)

In [42]:
nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38414 entries, 0 to 38413
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   npi                         38414 non-null  int64  
 1   entity_type_code            38414 non-null  float64
 2   provider_organization_name  7574 non-null   object 
 3   provider_last_name          30839 non-null  object 
 4   provider_first_name         30840 non-null  object 
 5   provider_middle_name        19914 non-null  object 
 6   provider_name_prefix        11560 non-null  object 
 7   provider_name_suffix        907 non-null    object 
 8   provider_credential         25323 non-null  object 
 9   address_01                  38414 non-null  object 
 10  address_02                  9957 non-null   object 
 11  city                        38414 non-null  object 
 12  state                       38414 non-null  object 
 13  zip_9                       384

In [51]:
# Take a look at the individual providers who do not have a primary specialty
nash_nppes[(nash_nppes['primary_taxonomy'].isna() == True)
           & (nash_nppes['entity_type_code'] == 1)]#.nunique() # Add this to count them

npi  entity_type_code provider_organization_name  \
258    1114923737               1.0                       None   
333    1801893631               1.0                       None   
1411   1215918321               1.0                       None   
1859   1477532448               1.0                       None   
1987   1346210457               1.0                       None   
2072   1215908058               1.0                       None   
2114   1780656777               1.0                       None   
2133   1245202266               1.0                       None   
2274   1588639561               1.0                       None   
2301   1558337097               1.0                       None   
2308   1073589305               1.0                       None   
2321   1659347557               1.0                       None   
2349   1346217999               1.0                       None   
2356   1992772206               1.0                       None   
2406   1821066341               1.0                       None   
2471   1306815576               1.0                       None   
2512   1831158112               1.0                       None   
2603   1598727869               1.0                       None   
2629   1134181720               1.0                       None   
2691   1568425213               1.0                       None   
2750   1336104645               1.0                       None   
2796   1841256112               1.0                       None   
3531   1528006707               1.0                       None   
3537   1386682318               1.0                       None   
3547   1013955707               1.0                       None   
4079   1801832944               1.0                       None   
4129   1437185519               1.0                       None   
4166   1811923246               1.0                       None   
4188   1629005582               1.0                       None   
4197   1962439877               1.0                       None   
4205   1417984964               1.0                       None   
4309   1386673978               1.0                       None   
4462   1053344549               1.0                       None   
4654   1982629952               1.0                       None   
5008   1245253277               1.0                       None   
5083   1316961279               1.0                       None   
5119   1255345385               1.0                       None   
5363   1316955040               1.0                       None   
5746   1194830612               1.0                       None   
5753   1770698425               1.0                       None   
5754   1679688329               1.0                       None   
5833   1013025766               1.0                       None   
5876   1235247339               1.0                       None   
6085   1336241777               1.0                       None   
6125   1720181563               1.0                       None   
6231   1225133267               1.0                       None   
6242   1003912486               1.0                       None   
6306   1891891883               1.0                       None   
6375   1053419382               1.0                       None   
6552   1851491948               1.0                       None   
6661   1992896435               1.0                       None   
7003   1699860056               1.0                       None   
7457   1659469682               1.0                       None   
7519   1538258926               1.0                       None   
7575   1598855934               1.0                       None   
7696   1093897704               1.0                       None   
7927   1457436321               1.0                       None   
8007   1649358581               1.0                       None   
8060   1841378155               1.0                       None   
8061   1982782207               1.0                   

*There are 106 individual providers who do not have a specialty. At a quick glance, there is no clear connection between these providers - they are in different areas, have different degrees, etc.*

In [66]:
# Take a look at the types of organizations in the dataset
# There are 152 unique classifications plus null values
nash_nppes[nash_nppes['entity_type_code'] == 2].groupby('Classification')['npi'].count().sort_values(ascending = False)

Classification
Clinic/Center                                                                                     981
Pharmacy                                                                                          559
Dentist                                                                                           519
Internal Medicine                                                                                 398
Durable Medical Equipment & Medical Supplies                                                      380
Chiropractor                                                                                      268
Family Medicine                                                                                   260
Specialist                                                                                        228
In Home Supportive Care                                                                           188
Nurse Practitioner                                                 

In [67]:
# Count the number of organizations that do not have a primary specialty
nash_nppes[(nash_nppes['primary_taxonomy'].isna() == True)
           & (nash_nppes['entity_type_code'] == 2)].nunique() # Add this to count them
# There are 92 organizations that do not have a specialty

npi                           92
entity_type_code               1
provider_organization_name    70
provider_last_name             0
provider_first_name            0
provider_middle_name           0
provider_name_prefix           0
provider_name_suffix           0
provider_credential            0
address_01                    86
address_02                    21
city                          20
state                          1
zip_9                         85
zip_5                         34
primary_taxonomy               0
Classification                 0
dtype: int64

### Follow just one NPI through the tables

In [76]:
sample = """
WITH nashville_providers AS (
    SELECT n.*
        , s.Classification AS specialty

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
)

SELECT n.npi
    , n.specialty
    , rto.*
    , rfrom.*
    
FROM nashville_providers AS n
    
LEFT JOIN nashville_referrals AS rto
    ON rto.to_npi = n.npi
    
LEFT JOIN nashville_referrals AS rfrom
    ON rfrom.from_npi = n.npi

WHERE npi = 1215064852
"""

In [77]:
liz_null = pd.read_sql(sample, db)

In [75]:
liz_null

npi               specialty     index    from_npi      to_npi  \
0   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
1   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
2   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
3   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
4   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
5   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
6   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
7   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
8   1215064852  Psychiatry & Neurology   1777418  1073531042  1215064852   
9   1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
10  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
11  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
12  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
13  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
14  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
15  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
16  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
17  1215064852  Psychiatry & Neurology   2732631  1326130642  1215064852   
18  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
19  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
20  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
21  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
22  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
23  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
24  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
25  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
26  1215064852  Psychiatry & Neurology  12565235  1467460725  1215064852   
27  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
28  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
29  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
30  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
31  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
32  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
33  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
34  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
35  1215064852  Psychiatry & Neurology  15967368  1861479545  1215064852   
36  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
37  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
38  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
39  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
40  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
41  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
42  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
43  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
44  1215064852  Psychiatry & Neurology  25617535  1295726032  1215064852   
45  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
46  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
47  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
48  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
49  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
50  1215064852  Psychiatry & Neurology  28491271  1578591756  1215064852   
51  1215064852  Psychiatry & Neurology  28491271  157859175

In [ ]:
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")

### Take a look at the total hop-teaming data joined up

In [40]:
# Write a query 
query = """
WITH nashville_providers AS (
    SELECT n.*

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
)

SELECT COUNT(*) AS tot_rows
    , COUNT(DISTINCT n.npi) AS unique_npis
    , COUNT(DISTINCT(CASE WHEN s.npi IS NOT NULL THEN s.npi ELSE NULL END)) AS npis_with_specialty
    , COUNT(DISTINCT(CASE WHEN rto.to_npi IS NOT NULL THEN rto.to_npi ELSE NULL END)) AS npis_who_refer_out
    , COUNT(DISTINCT(CASE WHEN rfrom.to_npi IS NOT NULL THEN rfrom.to_npi ELSE NULL END)) AS npis_who_get_referrals
    
FROM nashville_providers AS n

LEFT JOIN specialty AS s
    ON s.npi = n.npi
    
LEFT JOIN nashville_referrals AS rto
    ON rto.to_npi = n.npi
    
LEFT JOIN nashville_referrals AS rfrom
    ON rfrom.from_npi = n.npi
"""

In [ ]:
nashville_providers = pd.read_sql(query, db)

In [ ]:
nashville_providers